### 얼굴 감정 인식

In [1]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Conv2D, MaxPooling2D, Flatten, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint

In [11]:
# 이미지 데이터 제네레이터를 만들기
train_generator = ImageDataGenerator(rotation_range=10, # 랜덤 회전 각도
                                     zoom_range=0.2, # 줌인 비율
                                     horizontal_flip=True, # 가로 반전
                                     rescale=1/255) # 정규화 작업

train_dataset = train_generator.flow_from_directory(directory='C:/Users/main/workspace/vision/2025-06-11/Fer_2013/train',
                                                    target_size=(48,48),
                                                    class_mode='categorical',
                                                    batch_size=16,
                                                    shuffle=True,
                                                    seed=10)

test_generator = ImageDataGenerator(rescale=1/255)
test_dataset = test_generator.flow_from_directory(directory='C:/Users/main/workspace/vision/2025-06-11/Fer_2013/test',
                                                  target_size=(48,48),
                                                  class_mode='categorical',
                                                  batch_size=1,
                                                  shuffle=False,
                                                  seed=10)

# FER 데이터셋은 7가지의 감정 분류를 가짐, 디텍터는 32개 이미지는 48X48 셋을 가지고 있음
num_classes = 7
num_detectors = 32
width, height = 48, 48

# 이를 반영하는 망을 만들어 보도록 하자
network = Sequential()
network.add(Conv2D(filters=num_detectors, kernel_size=3, activation='relu', padding='same', input_shape=(width, height, 3)))
network.add(BatchNormalization())
network.add(Conv2D(filters=num_detectors, kernel_size=3, activation='relu', padding='same'))
network.add(BatchNormalization())
network.add(MaxPooling2D(pool_size=(2, 2)))
network.add(Dropout(0.2))
network.add(Conv2D(2*num_detectors, 3, activation='relu', padding='same'))
network.add(BatchNormalization())
network.add(Conv2D(2*num_detectors, 3, activation='relu', padding='same'))
network.add(BatchNormalization())
network.add(MaxPooling2D(pool_size=(2, 2)))
network.add(Dropout(0.2))
network.add(Conv2D(2*2*num_detectors, 3, activation='relu', padding='same'))
network.add(BatchNormalization())
network.add(Conv2D(2*2*num_detectors, 3, activation='relu', padding='same'))
network.add(BatchNormalization())
network.add(MaxPooling2D(pool_size=(2, 2)))
network.add(Dropout(0.2))
network.add(Conv2D(2*2*2*num_detectors, 3, activation='relu', padding='same'))
network.add(BatchNormalization())
network.add(Conv2D(2*2*2*num_detectors, 3, activation='relu', padding='same'))
network.add(BatchNormalization())
network.add(MaxPooling2D(pool_size=(2, 2)))
network.add(Dropout(0.2))
network.add(Flatten())
network.add(Dense(2*2*num_detectors, activation='relu'))
network.add(BatchNormalization())
network.add(Dropout(0.2))
network.add(Dense(2*num_detectors, activation='relu'))
network.add(BatchNormalization())
network.add(Dropout(0.2))
network.add(Dense(num_classes, activation='softmax'))
network.summary()


# 이제 네트워크를 컴파일 하기
network.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])

# 모델 학습시 오버피팅을 방지하기 위해 정확도를 모니터링하여 조기에 학습을 종료시키는 인스턴스 정의 
monitor_val_acc = EarlyStopping(monitor='val_accuracy', patience=5)
# loss 제일 낮을 때 가중치 저장
filename = 'emotion_best.h5'
checkpoint = ModelCheckpoint(filename,             # file명을 지정합니다
                             verbose=1,            # 로그를 출력합니다
                             save_best_only=True   # 가장 best 값만 저장합니다
                            )
epochs = 70
history= network.fit(train_dataset, epochs=epochs, validation_data=test_dataset, callbacks=[checkpoint, monitor_val_acc]).history
print('학습종료!')
network.save(filename)

score = network.evaluate(test_dataset)
print('Test loss:', score[0]) 
print('Test accuracy:', score[1]*100)



Found 28709 images belonging to 7 classes.
Found 7178 images belonging to 7 classes.
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 48, 48, 32)        896       
                                                                 
 batch_normalization (Batch  (None, 48, 48, 32)        128       
 Normalization)                                                  
                                                                 
 conv2d_1 (Conv2D)           (None, 48, 48, 32)        9248      
                                                                 
 batch_normalization_1 (Bat  (None, 48, 48, 32)        128       
 chNormalization)                                                
                                                                 
 max_pooling2d (MaxPooling2  (None, 24, 24, 32)        0         
 D)                                  

c:\Users\main\miniconda3\envs\deepface\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


1795/1795 [==============================] - ETA: 0s - loss: 1.4996 - accuracy: 0.4179
Epoch 2: val_loss improved from 1.48752 to 1.34732, saving model to emotion_best.h5
1795/1795 [==============================] - 304s 170ms/step - loss: 1.4996 - accuracy: 0.4179 - val_loss: 1.3473 - val_accuracy: 0.4812
Epoch 3/70
1795/1795 [==============================] - ETA: 0s - loss: 1.3629 - accuracy: 0.4792
Epoch 3: val_loss improved from 1.34732 to 1.27785, saving model to emotion_best.h5
1795/1795 [==============================] - 295s 165ms/step - loss: 1.3629 - accuracy: 0.4792 - val_loss: 1.2778 - val_accuracy: 0.5064
Epoch 4/70
1795/1795 [==============================] - ETA: 0s - loss: 1.2915 - accuracy: 0.5127
Epoch 4: val_loss improved from 1.27785 to 1.17879, saving model to emotion_best.h5
1795/1795 [==============================] - 584s 325ms/step - loss: 1.2915 - accuracy: 0.5127 - val_loss: 1.1788 - val_accuracy: 0.5463
Epoch 5/70
1795/1795 [==============================] 

KeyboardInterrupt: 

In [ ]:
import pandas as pd #데이터를 분석 및 조작하기 위한 소프트웨어 라이브러리
import matplotlib.pyplot as plt #다양한 데이터를 많은 방법으로 도식화 할 수 있도록 하는 파이썬 라이브러리

plt.figure(figsize=(16,5)) 
# 만들어진 모델에 대해 train dataset과 validation dataset의 loss 를 그래프로 표현
plt.subplot(1, 2, 1) 
plt.plot(history['loss']) 
plt.plot(history['val_loss']) 
plt.title('model loss') 
plt.ylabel('loss') 
plt.xlabel('epoch') 
plt.legend(['train', 'validation'], loc='upper left')

# 만들어진 모델에 대해 train dataset과 validation dataset의 accuracy 를 그래프로 표현
plt.subplot(1, 2, 2) 
plt.plot(history['accuracy']) 
plt.plot(history['val_accuracy']) 
plt.title('model accuracy') 
plt.ylabel('accuracy') 
plt.xlabel('epoch') 
plt.legend(['train', 'validation'], loc='upper left')


In [ ]:
import numpy as np
import tensorflow as tf
from keras.models import load_model
from tensorflow.keras.preprocessing.image import ImageDataGenreator
import os
import cv2
import dlib

In [ ]:
test_generator = ImageDagaGenerator(rescale=1/255)
train_dataset = train_generator.flow_from_directory(directory='C:/Users/main/workspace/vision/2025-06-11/Fer_2013/train',
                                                    target_size=(48,48),
                                                    class_mode='categorical',
                                                    batch_size=1,
                                                    shuffle=True,
                                                    seed=10)

network = load_model('emotion_best.h5')

image_list= []
test_images = os.path.join(os.getcwd(), 'test_list')
face_detector = dlib.cnn_face_detection_model_v1('./files/mmod_human_face_detector.dat')

for root, dirs, files in os.walk(test_images):
    for file in files :
        if file.endswith('jpeg') or file.endswith('jpg') or file.endswith('png')
        image_path = os.path.join(test_images, file)
        print(image_path)
        image_list.append(cv2.imread(image_path))
        
        for img in image_list:
            gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            faces = face_detector(img, 1)
            for face_detection in faces:
                left, top = face_detection.rect.left(), face_detection.rect.top()
                right, bottom = face_detection.rect.right(), face_detection.rect.bottom()
                roi = img[top:bottom, left:right]
                roi = cv2.resize(roi, (48, 48))
                roi = roi / 255
                roi = np.expand_dims(roi, axis=0)
                pred_probability = network.predict(roi)
                print(pred_probability)
                print(np.argmax(pred_probability))
                print(test_dataset.class_indices)
                i = 0
                name = ""
                for index in test_dataset.class_indices:
                    print(index)
                    if i == np.argmax(pred_probability):
                        name = index
                    i += 1
                    
                    font = cv2.FONT_HERSHEY_SIMPLEX
                    cv2.putText(img, name, (left,top), font, 1, (0,0,255), 2)
                    cv2.rectangle(img, (left,top), (right, bottom), (0,255,0), 2)
                    
            cv2.imshow('Preview', img)
            if cv2.waitKey(0) >= 0 :
                continue
            
cv2.destoryAllWindows()